In [1]:
from langchain.utilities.duckduckgo_search import DuckDuckGoSearchAPIWrapper
import yfinance
import json

def get_ticker(inputs):
    ddg = DuckDuckGoSearchAPIWrapper()
    company_name = inputs["company_name"]
    return ddg.run(f"Ticker symbol of {company_name}")

def get_income_statement(inputs):
    ticker = inputs["ticker"]
    stock = yfinance.Ticker(ticker)
    return json.dumps(stock.income_stmt.to_json())

def get_balance_sheet(inputs):
    ticker = inputs["ticker"]
    stock = yfinance.Ticker(ticker)
    return json.dumps(stock.balance_sheet.to_json())

def get_daily_stock_performance(inputs):
    ticker = inputs["ticker"]
    stock = yfinance.Ticker(ticker)
    return json.dumps(stock.history(period="3mo").to_json())

functions_map = {
    "get_ticker": get_ticker,
    "get_income_statement": get_income_statement,
    "get_balance_sheet": get_balance_sheet,
    "get_daily_stock_performance": get_daily_stock_performance,
}

functions = [
    {
        "type": "function",
        "function": {
            "name": "get_ticker",
            "description": "Given the name of a company returns its ticker symbol",
            "parameters": {
                "type": "object",
                "properties":{
                    "company_name": {
                        "type":"string",
                        "description":"The name of the company"
                    }
                },
                "required": ["company_name"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "get_income_statement",
            "description": "Given a ticker symbol (i.e AAPL) returns the company's income statement.",
            "parameters": {
                "type": "object",
                "properties": {
                    "ticker": {
                        "type": "string",
                        "description": "Ticker symbol of the company",
                    },
                },
                "required": ["ticker"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "get_balance_sheet",
            "description": "Given a ticker symbol (i.e AAPL) returns the company's balance sheet.",
            "parameters": {
                "type": "object",
                "properties": {
                    "ticker": {
                        "type": "string",
                        "description": "Ticker symbol of the company",
                    },
                },
                "required": ["ticker"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "get_daily_stock_performance",
            "description": "Given a ticker symbol (i.e AAPL) returns the performance of the stock for the last 100 days.",
            "parameters": {
                "type": "object",
                "properties": {
                    "ticker": {
                        "type": "string",
                        "description": "Ticker symbol of the company",
                    },
                },
                "required": ["ticker"],
            },
        },
    },
]

# get_income_statement({"ticker":"AAPL"})

In [2]:
from openai import OpenAI

client = OpenAI()

# assistant = client.beta.assistants.create(
# name="Investor Assistant",
# instructions="You help users do research on publicly traded companies and you help users decide if they should buy the stock or not.",
# tools=functions,
# model="gpt-4o",
# )

assistant_id = "asst_13eo2MdAQaNGc4XTBhTHlOW3"



In [3]:
client = OpenAI()

thread = client.beta.threads.create(
    messages = [
        {
            "role":"user",
            #"content":"I want to know if the Salesforce stock is a good buy"
            "content":"I want to know if the Cloudflare stock is a good buy"
            
        }
    ]
)

thread

Thread(id='thread_D74EWqRlN3FxnPMdvVaTRtUd', created_at=1727923274, metadata={}, object='thread', tool_resources=ToolResources(code_interpreter=None, file_search=None), _request_id='req_a50bf620c37253cc2c9ddb1c67235cc6', __exclude_fields__={'_request_id', '__exclude_fields__'})

In [4]:

run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant_id
)
run

Run(id='run_P3oPEB462gkmMrJ8tgVSxZvP', assistant_id='asst_13eo2MdAQaNGc4XTBhTHlOW3', cancelled_at=None, completed_at=None, created_at=1727923280, expires_at=1727923880, failed_at=None, incomplete_details=None, instructions='You help users do research on publicly traded companies and you help users decide if they should buy the stock or not.', last_error=None, max_completion_tokens=None, max_prompt_tokens=None, metadata={}, model='gpt-4o', object='thread.run', parallel_tool_calls=True, required_action=None, response_format='auto', started_at=None, status='queued', thread_id='thread_D74EWqRlN3FxnPMdvVaTRtUd', tool_choice='auto', tools=[FunctionTool(function=FunctionDefinition(name='get_ticker', description='Given the name of a company returns its ticker symbol', parameters={'type': 'object', 'properties': {'company_name': {'type': 'string', 'description': 'The name of the company'}}, 'required': ['company_name']}, strict=False), type='function'), FunctionTool(function=FunctionDefinition(

In [5]:
def get_run(run_id, thread_id):
    return client.beta.threads.runs.retrieve(
        run_id=run_id,
        thread_id=thread_id
    )

def send_messages(thread_id, content):
    return client.beta.threads.messages.create(
        thread_id=thread_id,
        role="user",
        content=content
    )

def get_messages(thread_id):
    messages = client.beta.threads.messages.list(
        thread_id=thread_id
    )
    messages = list(messages)
    messages.reverse()
    for message in messages:
        print(f"{message.role}: {message.content[0].text.value}")

def get_tool_outputs(run_id, thread_id):
    run = get_run(run_id, thread_id)
    outputs = []
    for action in run.required_action.submit_tool_outputs.tool_calls:
        #print(action)
        action_id = action.id
        function = action.function
        print(f"Calling function: {function.name} with arg {function.arguments}")
        outputs.append({
            "output":functions_map[function.name](json.loads(function.arguments)),
            "tool_call_id":action_id
        })
    return outputs

def submit_tool_outputs(run_id, thread_id):
    outputs = get_tool_outputs(run_id, thread_id)
    return client.beta.threads.runs.submit_tool_outputs(
        run_id=run_id,
        thread_id=thread_id,
        tool_outputs=outputs
    )

In [16]:
get_run(run.id, thread.id).status

'completed'

In [17]:
get_messages(thread.id)

user: I want to know if the Cloudflare stock is a good buy
assistant: To evaluate whether Cloudflare (Ticker: NET) is a good buy, let's analyze its financial metrics and recent stock performance:

### Financial Overview
#### Income Statement Highlights (for the most recent period):
- **Total Revenue**: $1,296,745,000
- **Net Income**: -$183,949,000
- **Gross Profit**: $989,740,000
- **Operating Income**: -$185,485,000
- **EBITDA**: -$36,170,000
- **Basic/Diluted EPS**: -$0.55

#### Balance Sheet Highlights:
- **Total Assets**: $2,759,767,000
- **Total Liabilities**: $1,996,720,000
- **Stockholders' Equity**: $763,047,000
- **Net Debt**: $1,196,498,000

### Recent Stock Performance (Last 100 Days):
- **Recent Price Range**: ~$76 to ~$86
- **Most Recent Closing Price**: $79.34 (tendency to fluctuate with market conditions)
- **No Dividends or Stock Splits**: Cloudflare has not paid dividends or had stock splits in the recent period

### Analysis
- **Revenue Growth**: The company has show

In [12]:
get_tool_outputs(run.id, thread.id)

Calling function: get_income_statement with arg {"ticker": "NET"}
Calling function: get_balance_sheet with arg {"ticker": "NET"}
Calling function: get_daily_stock_performance with arg {"ticker": "NET"}


[{'output': '"{\\"1703980800000\\":{\\"Tax Effect Of Unusual Items\\":-10563000.0,\\"Tax Rate For Calcs\\":0.21,\\"Normalized EBITDA\\":14130000.0,\\"Total Unusual Items\\":-50300000.0,\\"Total Unusual Items Excluding Goodwill\\":-50300000.0,\\"Net Income From Continuing Operation Net Minority Interest\\":-183949000.0,\\"Reconciled Depreciation\\":135820000.0,\\"Reconciled Cost Of Revenue\\":307005000.0,\\"EBITDA\\":-36170000.0,\\"EBIT\\":-171990000.0,\\"Net Interest Income\\":62295000.0,\\"Interest Expense\\":5872000.0,\\"Interest Income\\":68167000.0,\\"Normalized Income\\":-144212000.0,\\"Net Income From Continuing And Discontinued Operation\\":-183949000.0,\\"Total Expenses\\":1482230000.0,\\"Total Operating Income As Reported\\":-185485000.0,\\"Diluted Average Shares\\":333656000.0,\\"Basic Average Shares\\":333656000.0,\\"Diluted EPS\\":-0.55,\\"Basic EPS\\":-0.55,\\"Diluted NI Availto Com Stockholders\\":-183949000.0,\\"Net Income Common Stockholders\\":-183949000.0,\\"Net Incom

In [13]:
submit_tool_outputs(run.id, thread.id)

Calling function: get_income_statement with arg {"ticker": "NET"}
Calling function: get_balance_sheet with arg {"ticker": "NET"}
Calling function: get_daily_stock_performance with arg {"ticker": "NET"}


Run(id='run_P3oPEB462gkmMrJ8tgVSxZvP', assistant_id='asst_13eo2MdAQaNGc4XTBhTHlOW3', cancelled_at=None, completed_at=None, created_at=1727923280, expires_at=1727923880, failed_at=None, incomplete_details=None, instructions='You help users do research on publicly traded companies and you help users decide if they should buy the stock or not.', last_error=None, max_completion_tokens=None, max_prompt_tokens=None, metadata={}, model='gpt-4o', object='thread.run', parallel_tool_calls=True, required_action=None, response_format='auto', started_at=1727923306, status='queued', thread_id='thread_D74EWqRlN3FxnPMdvVaTRtUd', tool_choice='auto', tools=[FunctionTool(function=FunctionDefinition(name='get_ticker', description='Given the name of a company returns its ticker symbol', parameters={'type': 'object', 'properties': {'company_name': {'type': 'string', 'description': 'The name of the company'}}, 'required': ['company_name']}, strict=False), type='function'), FunctionTool(function=FunctionDefin

In [ ]:
# 메세지를 보내고 확인하는건 get_messages로
send_messages(thread.id, "Now I want to know if Cloudflare is a good buy.") 